# Final Project: Decide on global dual sourcing strategy

team: \
David Yang \
Jack Chen \
Joyce Wu

In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('final project 2024.csv')

In [4]:
data.head(10)

,period,demand
0,1,21
1,2,81
2,3,32
3,4,58
4,5,47
5,6,49
6,7,66
7,8,29
8,9,55
9,10,39


In [5]:
s = 10 # sales price
c_m = 8 # cost mexico
c_c = 7.25 # cost china
i = 0.01 # interest rate
ini_bal = 0 # initial balance

## Single Sourcing: China

### Cumulative Average Forecast

Without optimization

In [28]:
def single_source_china_cum_avg(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance):
    cash_balance = initial_cash_balance
    inventory = 0
    orders = []  # List to keep track of orders placed but not yet received

    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]

        # Calculate cumulative average demand up to the current period
        if period > 1:
            cum_avg_demand = np.ceil(np.mean(data.loc[data['period'] < period, 'demand']))
        else:
            cum_avg_demand = np.ceil(current_demand)  # For the first period, use the current demand

        beginning_inventory = inventory  # Record beginning inventory

        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0

        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Determine order quantity based on cumulative average demand and current inventory
        if cum_avg_demand >= inventory:
            order_quantity = int(cum_avg_demand - inventory)
        else:
            order_quantity = 0

        if order_quantity > 0:
            orders.append((period + 4, order_quantity))  # Place order to be received in 4 periods
            order_cost = order_quantity * sourcing_cost
            cash_balance -= order_cost

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        # Record ending inventory before new orders arrive
        ending_inventory = inventory

        # Check if any orders are arriving in this period
        orders_to_receive = [order for order in orders if order[0] == period]
        for order in orders_to_receive:
            inventory += order[1]
            orders.remove(order)

        # Debugging outputs
        '''
        print(f"Period: {period}")
        print(f"Beginning Inventory: {beginning_inventory}")
        print(f"Current Demand: {current_demand}")
        print(f"Sales: {sales}")
        print(f"Revenue: {revenue}")
        print(f"Order Quantity: {order_quantity}")
        print(f"Order Cost: {order_cost if order_quantity > 0 else 0}")
        print(f"Ending Inventory: {ending_inventory}")
        print(f"Profit from Operation: {revenue - (order_cost if order_quantity > 0 else 0)}")
        print(f"Profit from Interest: {cash_balance_ai - cash_balance}")
        print(f"Cash Balance: {cash_balance}")
        print("-" * 40)
        '''

    return cash_balance

# Calculate ending bank account value for sourcing from China
ending_bank_account_value_china = single_source_china_cum_avg(data, s, c_c, i, ini_bal)
print(f"Ending Bank Account Value:${ending_bank_account_value_china:.2e}")

Ending Bank Account Value:$1.32e+47


With optimization

In [27]:
def single_source_china_cum_avg(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, sl):
    cash_balance = initial_cash_balance
    inventory = 0
    errors = []
    orders = []  # List to keep track of orders placed but not yet received
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        
        # Calculate cumulative average demand up to the current period
        if period > 1:
            cum_avg_demand = np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))
        else:
            cum_avg_demand = np.ceil(current_demand)  # For the first period, use the current demand
        
        beginning_inventory = inventory
        
        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0

        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue
        
        # Determine order quantity based on cumulative average demand and current inventory
        order_quantity = 0
        order_cost = 0
        
        # Calculate error and update errors list
        forecast_error = current_demand - cum_avg_demand
        errors.append(forecast_error)
            
        # Calculate the specified percentile of errors
        if len(errors) > 0:
            error_percentile = np.percentile(errors, sl)
        else:
            error_percentile = 0

        # Adjust order quantity based on forecast and error percentile
        adjusted_forecast = cum_avg_demand + error_percentile
        if adjusted_forecast >= inventory:
            order_quantity = int(adjusted_forecast - inventory)
        else:
            order_quantity = 0

        if order_quantity > 0:
            orders.append((period + 4, order_quantity))  # Place order to be received in 4 periods
            order_cost = order_quantity * sourcing_cost
            cash_balance -= order_cost
        current_cash = cash_balance  # record current balance
            
        ending_inventory = inventory

        # Check if any orders are arriving in this period
        orders_to_receive = [order for order in orders if order[0] == period]
        for order in orders_to_receive:
            inventory += order[1]
            orders.remove(order)

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)
        
        # Debugging outputs (commented out)
        """
        print(f"Period: {period}")
        print(f"Beginning Inventory: {beginning_inventory}")
        print(f"Current Demand: {current_demand}")
        print(f"Sales: {sales}")
        print(f"Revenue: {revenue}")
        print(f"Order Quantity: {order_quantity}")
        print(f"Order Cost: {order_cost}")
        print(f"Ending Inventory: {ending_inventory}")
        print(f"Profit from Operation: {revenue - order_cost}")
        print(f"Profit from Interest: {cash_balance - current_cash}")
        print(f"Cash Balance: {cash_balance}")
        print("-" * 40)
        """
    
    return cash_balance

# Define range of service levels to test (percentile thresholds)
percentile_thresholds = np.linspace(1, 99, 99)

# Optimization function to find the best service level
def optimize_service_level(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, percentile_thresholds):
    best_sl = None
    best_value = -np.inf
    
    for sl in percentile_thresholds:
        ending_bank_account_value = single_source_china_cum_avg(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, sl)
        if ending_bank_account_value > best_value:
            best_value = ending_bank_account_value
            best_sl = sl
    
    return best_sl, best_value

# Optimize service level
best_sl, best_value = optimize_service_level(data, s, c_c, i, ini_bal, percentile_thresholds)

# Print the best service level and the corresponding ending bank account value
print(f"Best Service Level: {best_sl:.0f}")
print(f"Ending Bank Account Value: ${best_value:.2e}")

Best Service Level: 53
Ending Bank Account Value: $1.32e+47


### Moving Average Forecast

Without optimization

In [29]:
def single_source_china_mov_avg(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, moving_avg_window, percentile_threshold):
    cash_balance = initial_cash_balance
    inventory = 0
    errors = []
    orders = []  # List to keep track of orders placed but not yet received

    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]

        # Calculate moving average demand over the specified window
        if period > moving_avg_window:
            moving_avg_demand = np.ceil(np.mean(data.loc[(data['period'] >= period - moving_avg_window) & (data['period'] < period), 'demand']))
        else:
            moving_avg_demand = np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))

        beginning_inventory = inventory  # Record beginning inventory
        
        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Initialize order quantity and cost
        order_quantity = 0
        order_cost = 0
        
        # Determine order quantity based on moving average demand and current inventory
        if period > moving_avg_window:
            # Calculate error and update errors list
            forecast_error = current_demand - moving_avg_demand
            errors.append(forecast_error)
            
            # Calculate the 95th percentile of errors
            if len(errors) > 0:
                error_percentile = np.percentile(errors, percentile_threshold)
            else:
                error_percentile = 0

            # Adjust order quantity based on forecast and error percentile
            adjusted_forecast = moving_avg_demand + error_percentile
            if adjusted_forecast >= inventory:
                order_quantity = int(adjusted_forecast - inventory)
            else:
                order_quantity = 0

            if order_quantity > 0:
                orders.append((period + 4, order_quantity))  # Place order to be received in 4 periods
                order_cost = order_quantity * sourcing_cost
                cash_balance -= order_cost
            
        # Record ending inventory before the new order arrives
        ending_inventory = inventory

        # Check if any orders are arriving in this period
        orders_to_receive = [order for order in orders if order[0] == period]
        for order in orders_to_receive:
            inventory += order[1]
            orders.remove(order)

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        # Debugging outputs (commented out)
        '''
        print(f"Period: {period}")
        print(f"Beginning Inventory: {beginning_inventory}")
        print(f"Current Demand: {current_demand}")
        print(f"Sales: {sales}")
        print(f"Revenue: {revenue}")
        print(f"Order Quantity: {order_quantity}")
        print(f"Order Cost: {order_cost}")
        print(f"Ending Inventory: {ending_inventory}")
        print(f"Profit from Operation: {revenue - order_cost}")
        print(f"Profit from Interest: {cash_balance_ai - cash_balance}")
        print(f"Cash Balance: {cash_balance_ai}")
        print(f"Forecast Error: {forecast_error}")
        print(f"95th Percentile of Errors: {error_percentile}")
        print(f"Adjusted Forecast: {adjusted_forecast}")
        print("-" * 40)
        '''
    
    return cash_balance

# Calculate ending bank account value with specified moving average window for China
window = 10
SL = 95
ending_bank_account_value_china = single_source_china_mov_avg(data, s, c_c, i, ini_bal, window, SL)
print(f"Ending Bank Account Value: ${ending_bank_account_value_china:.2e}")


Ending Bank Account Value: $1.14e+47


With optimization

In [30]:
def single_source_china_mov_avg(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, moving_avg_window, percentile_threshold):
    cash_balance = initial_cash_balance
    inventory = 0
    errors = []
    orders = []
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]

        # Calculate moving average demand over the specified window
        if period >= moving_avg_window:
            moving_avg_demand = np.ceil(np.mean(data.loc[(data['period'] > period - moving_avg_window) & (data['period'] <= period), 'demand']))
        else:
            moving_avg_demand = np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))

        beginning_inventory = inventory  # Record beginning inventory
        
        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Initialize order quantity and cost
        order_quantity = 0
        order_cost = 0
        
        # Determine order quantity based on moving average demand and current inventory
        if period >= moving_avg_window:
            # Calculate error and update errors list
            forecast_error = current_demand - moving_avg_demand
            errors.append(forecast_error)
            
            # Calculate the percentile of errors
            if len(errors) > 0:
                error_percentile = np.percentile(errors, percentile_threshold)
            else:
                error_percentile = 0

            # Adjust order quantity based on forecast, error percentile, and service level
            adjusted_forecast = moving_avg_demand + error_percentile
            reorder_point = adjusted_forecast * service_level / 100
            if inventory < reorder_point:
                order_quantity = int(adjusted_forecast - inventory)
            else:
                order_quantity = 0

            if order_quantity > 0:
                orders.append((period + 4, order_quantity))  
                order_cost = order_quantity * sourcing_cost
                cash_balance -= order_cost
            
            # Record ending inventory before the new order arrives
            ending_inventory = inventory
            orders_to_receive = [order for order in orders if order[0] == period]
            for order in orders_to_receive:
                inventory += order[1]
                orders.remove(order)

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        # Debugging outputs (commented out)
        '''
        print(f"Period: {period}")
        print(f"Beginning Inventory: {beginning_inventory}")
        print(f"Current Demand: {current_demand}")
        print(f"Sales: {sales}")
        print(f"Revenue: {revenue}")
        print(f"Order Quantity: {order_quantity}")
        print(f"Order Cost: {order_cost}")
        print(f"Ending Inventory: {ending_inventory}")
        print(f"Profit from Operation: {revenue - order_cost}")
        print(f"Profit from Interest: {cash_balance - current_cash}")
        print(f"Cash Balance: {cash_balance}")
        print(f"Forecast Error: {forecast_error}")
        print(f"Percentile of Errors: {error_percentile}")
        print(f"Adjusted Forecast: {adjusted_forecast}")
        print("-" * 40)
        '''
    
    return cash_balance

def optimize_service_level(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, moving_avg_window, service_level):
    best_service_level = None
    best_cash_balance = -np.inf
    
    for service_level in range(1, 101):
        cash_balance = single_source_china_mov_avg(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, moving_avg_window, service_level)
        if cash_balance > best_cash_balance:
            best_cash_balance = cash_balance
            best_service_level = service_level
    
    return best_service_level, best_cash_balance

window = 10

best_service_level, best_cash_balance = optimize_service_level(data, s, c_c, i, ini_bal, window, SL)

print(f"Best Service Level: {best_service_level}")
print(f"Ending Bank Account Value: ${best_cash_balance:.2e}")


Best Service Level: 99
Ending Bank Account Value: $1.17e+47


### Exponential Smoothing

Without optimizing SL

In [24]:
def single_source_china_exp_smt(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha, percentile_threshold):
    cash_balance = initial_cash_balance
    inventory = 0
    errors = []
    forecast = None 
    orders = []  
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        
        # Calculate exponential smoothing forecast
        if period == 1:
            forecast = current_demand  # Use the first period's demand as the initial forecast
        else:
            forecast = alpha * current_demand + (1 - alpha) * forecast

        # Calculate forecast error and update errors list
        if period > 1:
            forecast_error = current_demand - forecast
            errors.append(forecast_error)
            
            # Calculate the 95th percentile of errors
            if len(errors) > 0:
                error_percentile = np.percentile(errors, percentile_threshold)
            else:
                error_percentile = 0

            # Adjust order quantity based on forecast and error percentile
            adjusted_forecast = forecast + error_percentile
            if adjusted_forecast >= inventory:
                order_quantity = int(adjusted_forecast - inventory)
            else:
                order_quantity = 0

            if order_quantity > 0:
                orders.append((period + 4, order_quantity))  
                order_cost = order_quantity * sourcing_cost
                cash_balance -= order_cost

        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        # Check if any orders are arriving in this period
        orders_to_receive = [order for order in orders if order[0] == period]
        for order in orders_to_receive:
            inventory += order[1]
            orders.remove(order)

        '''
        print(f"Period: {period}")
        print(f"Beginning Inventory: {inventory}")
        print(f"Current Demand: {current_demand}")
        print(f"Sales: {sales}")
        print(f"Revenue: {revenue}")
        print(f"Order Quantity: {order_quantity}")
        print(f"Order Cost: {order_cost if order_quantity > 0 else 0}")
        print(f"Ending Inventory: {inventory}")
        print(f"Profit from Operation: {revenue - (order_cost if order_quantity > 0 else 0)}")
        print(f"Profit from Interest: {cash_balance - current_cash}")
        print(f"Cash Balance: {cash_balance}")
        print(f"Forecast Error: {forecast_error}")
        print(f"95th Percentile of Errors: {error_percentile}")
        print(f"Adjusted Forecast: {adjusted_forecast}")
        print("-" * 40)
        '''
    
    return cash_balance

# Optimization function to find the best alpha
def optimize_alpha(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, percentile_threshold, alpha_values):
    best_alpha = None
    best_value = -np.inf
    
    for alpha in alpha_values:
        ending_bank_account_value = single_source_china_exp_smt(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha, percentile_threshold)
        if ending_bank_account_value > best_value:
            best_value = ending_bank_account_value
            best_alpha = alpha
    
    return best_alpha, best_value

# Define range of alpha values to test
alpha_values = np.linspace(0.01, 0.99, 99)
SL = 95
# Optimize alpha
best_alpha2, best_value2 = optimize_alpha(data, s, c_c, i, ini_bal, SL, alpha_values)
print(f"Best Alpha: {best_alpha2:.2f}")
print(f"Ending Bank Account Value: ${best_value2:.2e}")

Best Alpha: 0.02
Ending Bank Account Value: 1.14e+47


With optimizing SL

In [27]:
def single_source_china_exp_smt(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha, percentile_threshold, service_level):
    cash_balance = initial_cash_balance
    inventory = 0
    errors = []
    forecast = None 
    orders = []  
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        
        # Calculate exponential smoothing forecast
        if period == 1:
            forecast = current_demand  # Use the first period's demand as the initial forecast
        else:
            forecast = alpha * current_demand + (1 - alpha) * forecast

        # Calculate forecast error and update errors list
        if period > 1:
            forecast_error = current_demand - forecast
            errors.append(forecast_error)
            
            # Calculate the percentile of errors
            if len(errors) > 0:
                error_percentile = np.percentile(errors, percentile_threshold)
            else:
                error_percentile = 0

            # Adjust order quantity based on forecast, error percentile, and service level
            adjusted_forecast = forecast + error_percentile
            reorder_point = adjusted_forecast * service_level / 100
            if inventory < reorder_point:
                order_quantity = int(adjusted_forecast - inventory)
            else:
                order_quantity = 0

            if order_quantity > 0:
                orders.append((period + 4, order_quantity))  
                order_cost = order_quantity * sourcing_cost
                cash_balance -= order_cost

        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        # Check if any orders are arriving in this period
        orders_to_receive = [order for order in orders if order[0] == period]
        for order in orders_to_receive:
            inventory += order[1]
            orders.remove(order)

        '''
        print(f"Period: {period}")
        print(f"Beginning Inventory: {inventory}")
        print(f"Current Demand: {current_demand}")
        print(f"Sales: {sales}")
        print(f"Revenue: {revenue}")
        print(f"Order Quantity: {order_quantity}")
        print(f"Order Cost: {order_cost if order_quantity > 0 else 0}")
        print(f"Ending Inventory: {inventory}")
        print(f"Profit from Operation: {revenue - (order_cost if order_quantity > 0 else 0)}")
        print(f"Profit from Interest: {cash_balance - current_cash}")
        print(f"Cash Balance: {cash_balance}")
        print(f"Forecast Error: {forecast_error}")
        print(f"Percentile of Errors: {error_percentile}")
        print(f"Adjusted Forecast: {adjusted_forecast}")
        print("-" * 40)
        '''
    
    return cash_balance

def optimize_alpha_service_level(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha_values, service_levels):
    best_alpha = None
    best_service_level = None
    best_value = -np.inf
    
    for alpha in alpha_values:
        for service_level in service_levels:
            ending_bank_account_value = single_source_china_exp_smt(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha, 95, service_level)  # 使用95作为固定的percentile_threshold
            if ending_bank_account_value > best_value:
                best_value = ending_bank_account_value
                best_alpha = alpha
                best_service_level = service_level
    
    return best_alpha, best_service_level, best_value

alpha_values = np.linspace(0.01, 0.99, 99)
service_levels = range(1, 101)

best_alpha, best_service_level, best_value = optimize_alpha_service_level(data, s, c_c, i, ini_bal, alpha_values, service_levels)

print(f"Best Alpha: {best_alpha:.2f}")
print(f"Best Service Level: {best_service_level}")
print(f"Ending Bank Account Value: {best_value}")

Best Alpha: 0.02
Best Service Level: 99
Ending Bank Account Value: 1.135863796919137e+47
